# Cuaderno 15: Modelo con restricciones de corte para el ATSP

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Recordar que, dados: 
* un grafo dirigido **completo** $D=(V,A)$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$;

el *problema del agente viajero asimétrico (Asymmetric Traveling Salesman Problem, ATSP)* consiste en encontrar un circuito que visite **todos** los nodos de $D$ y que tenga el menor costo posible.

En el Cuaderno 14 revisamos un modelo de programación lineal entera para este problema. Un aspecto importante a considerar en el modelo fueron las *restricciones de eliminación de subtoures*, las cuales consisitían en requerir que, para cualquier subconjunto de nodos $W \subset V$, con $\emptyset \neq W \neq V$, el número de arcos seleccionados en la solución que tengan sus dos extremos dentro de $W$ sea menor o igual a $\card{W} - 1$.

Una manera alternativa de eliminar subciclos consiste en requerir que la solución contenga siempre un arco del corte entrante $\delta^{-}(W)$ o un arco del corte saliente $\delta^{+}(W)$ para cada $W \subset V$ con $\emptyset \neq W \neq V$. Estas restricciones se conocen como *restricciones de corte*. Los conjuntos de corte están definidos por:

\begin{align*}
\delta^{-}(W) &:= \{ (i,j) \in A \, : \, i \not\in W, j \in W\}, \\
\delta^{+}(W) &:= \{ (i,j) \in A \, : \, i \in W, j \not\in W\}. \\
\end{align*}


De esta manera, el problema del agente viajero asimétrico puede formularse como el siguiente programa lineal entero:

\begin{align*} 
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(j, i) \in A} x_{ji} = 1, \quad \forall i \in V,\\
&\sum_{(i, j) \in A} x_{ij} = 1, \quad \forall i \in V,\\ 
&\sum_{(i,j) \in \delta^{-}(W)} x_{ij} \geq 1, \quad \forall W \subset V, \emptyset \neq W \neq V,\\
& x_{ij} \in \{0, 1\}, \quad \forall (i, j) \in A.
\end{align*}

Las dos primeras familias de restricciones corresponden a las restricciones de grado para los nodos.

La tercera familia incluye las restricciones de corte para eliminación de subtoures.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los datos. Usaremos la función `randint` para generar puntos con coordenadas aleatorias en el rango {0,..,100}. El costo de un arco $(i,j)$ será igual a la distancia euclideana entre $i$ y $j$:

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math 

# número de nodos del grafo
n = 10

# conjunto de nodos del grafo
V = gp.tuplelist(range(1,n+1))

# coordenadas de los nodos
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()


Vamos a necesitar construir una restricción de corte para cada subconjunto de nodos $W \subset V$ tal que $\emptyset \neq W \neq V$. Con esta finalidad, definimos una función `powerset` empleando las funciones `chain` y `combinations` del módulo `itertools`.

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# ejemplo:
for W in powerset([1,2,3]):
    print(W)

Definimos ahora el objeto modelo, las variables y la función objetivo:

In [ ]:
# crear el objeto modelo
m = gp.Model('atsp-corte')

# crear las variables
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# crear la función objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)


Añadimos las restricciones de grado:

In [ ]:
# restricciones de grado saliente
m.addConstrs((x.sum(i,'*')  == 1 for i in V), 
                 "g_saliente")
    
# restricciones de grado entrante
m.addConstrs((x.sum('*', i)  == 1 for i in V), 
                 "g_entrante")

Para construir las restricciones de corte, usamos la función `powerset` para iterar sobre todos los subconjuntos de vértices $W$ tales que $\emptyset \neq W \neq V$. Para cada subconjunto, determinamos su complemento $W_c:= V \setminus W$. La expresión `x.sum(Wc, W)` construye la suma de las variables asociadas a arcos del corte entrante de $W$.

In [ ]:
# restricciones de corte para eliminar subtoures
for W in powerset(V):
    if len(W)!=0 and len(W)!=len(tuple(V)):
        Wc = [i for i in V if i not in W]
        m.addConstr(x.sum(Wc, W) >= 1, "corte_ent[{}]".format(W))

Escribimos el modelo a un archivo de texto:

In [ ]:
# escribir el modelo a un archivo
m.write('atsp.lp')

Finalmente, resolvemos el modelo y mostramos la solución:

In [ ]:
# calcular la solución óptima
m.optimize()

# escribir la solución
if m.SolCount > 0:
    # recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nTour óptimo:')
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{} -> {}'.format(i, j))


Por último, empleamos el módulo `matplotlib` y la función `dibujarTour` del Cuaderno 14 para graficar el tour de la solución.


In [ ]:
import matplotlib.pyplot as plt

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    display(plt.show())
    
# crear lista con los arcos seleccionados en la solución
L = gp.tuplelist([(i,j) for i,j in A if vx[i,j]>=0.9])

# recuperar el tour como un ordenamiento de los nodos
# empezar (arbitrariamente) en el nodo 1
T = [1]
# nodo actual:
i = 1
while True:
    # determinar sucesor de i
    (i,j) = L.select(i, '*').pop(0)
    # colocar sucesor en la lista del tour y actualizar i
    T.append(j)
    i = j
    # terminar cuando el nodo colocado sea 1
    if i==1: 
        break;
        
# graficar el tour
dibujarTour(coordx, coordy, T)    

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del agente viajero asimétrico (ATSP)

# Luis M. Torres (EPN 2022)

import gurobipy as gp
from gurobipy import GRB
import random as rm
import math
import matplotlib.pyplot as plt
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    plt.show()
    
# iniciar generador de numeros aleatorios
rm.seed(0)

# número de nodos del grafo
n = 10

# conjunto de nodos del grafo
V = gp.tuplelist(range(1,n+1))

# coordenadas de los nodos
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()

try:
    # crear el objeto modelo
    m = gp.Model('tsp')

    # crear las variables
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # crear la función objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    # restricciones de grado saliente
    m.addConstrs((x.sum(i,'*')  == 1 for i in V), 
                 "g_saliente")
    
    # restricciones de grado entrante
    m.addConstrs((x.sum('*', i)  == 1 for i in V), 
                 "g_entrante")

    # restricciones de corte para eliminar subtoures
    for W in powerset(V):
        if len(W)!=0 and len(W)!=len(V):
            Wc = [i for i in V if i not in W]
            m.addConstr(x.sum(Wc, W) >= 1, "subciclo[{}]".format(W))

    # escribir el modelo a un archivo
    # m.write('tsp.lp')

    # calcular la solución óptima
    m.optimize()

    # escribir la solución
    if m.SolCount > 0:
        # recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nTour optimo:')
        for i,j in A:
            if vx[i,j] >= 0.99:
                print('{} -> {}'.format(i, j))
                
        # crear lista con arcos seleccionados en la solución
        L = gp.tuplelist([(i,j) for i,j in A if vx[i,j]>=0.9])

        # recuperar el tour como un ordenamiento de los nodos
        T = [1]
        # nodo actual:
        i = 1
        while True:
            # determinar sucesor de i
            (i,j) = L.select(i, '*').pop(0)
            # colocar sucesor en la lista del tour y actualizar i
            T.append(j)
            i = j
            # terminar cuando el nodo colocado sea 1
            if i==1: break;
        
        # graficar el tour
        dibujarTour(coordx, coordy, T)    
    
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')